In [ ]:
import torch
import torchvision
import torch.nn as nn 
import torch.optim as optim 
import torch.nn.functional as F 
from torch.utils.data import DataLoader
import torchvision.datasets as datasets  
import torchvision.transforms as transforms
from torchvision.models import EfficientNet_B3_Weights

In [ ]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
num_classes = 10
learning_rate = 0.001
batch_size = 512
num_epochs = 20

# Simple Identity class that let's input pass without changes
class Identity(nn.Module):
    def __init__(self):
        super(Identity, self).__init__()

    def forward(self, x):
        return x


# Load pretrain model & modify it
pretrained_weight = EfficientNet_B3_Weights.IMAGENET1K_V1
model = torchvision.models.efficientnet_b3(weights=pretrained_weight)

# do finetuning then set requires_grad = False
# Remove these two lines if want to train entire model,
# and only want to load the pretrain weights.
# for param in model.parameters():
#     param.requires_grad = False

# model.avgpool = Identity()
model.classifier = nn.Sequential(
    nn.Dropout(p=0.3, inplace=True),
    nn.Linear(in_features=1536, out_features=10, bias=True))

model.to(device)


Downloading: "https://download.pytorch.org/models/efficientnet_b3_rwightman-cf984f9c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b3_rwightman-cf984f9c.pth


  0%|          | 0.00/47.2M [00:00<?, ?B/s]

EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 40, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(40, 40, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=40, bias=False)
            (1): BatchNorm2d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(40, 10, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(10, 40, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActiv

In [ ]:
# Load Data
tfm = transforms.Compose([transforms.ToTensor(), 
                          transforms.Normalize((0.49139968, 0.48215841, 0.44653091), 
                                               (0.24703223, 0.24348513, 0.26158784))])
train_dataset = datasets.CIFAR10(
    root="./data", train=True, download=True, transform=tfm)
train_loader = DataLoader(
    dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_dataset = datasets.CIFAR10(
    root='./data', train=False, download=True, transform=tfm)
val_loader = DataLoader(
    dataset=val_dataset, batch_size=batch_size*2, shuffle=False)
# Loss and optimizer
criterion = nn.CrossEntropyLoss() # F.cross_entropy also can
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate)

# data, targets = next(iter(train_loader))

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [ ]:
# Train
%%time
for epoch in range(num_epochs):
    model.train()
    losses = []

    for batch_idx, (data, targets) in enumerate(train_loader):
    # send data to cuda if possible
        data = data.to(device)
        targets = targets.to(device)

        # forward
        scores = model(data)
        loss = criterion(scores, targets)

        losses.append(loss.item())

        # backward
        loss.backward()

        # gradient descent or adam step
        optimizer.step()

        # set gradient back to zero
        optimizer.zero_grad()

    print(f"Epoch [{epoch}], Loss {sum(losses)/len(losses):.4f}")

# Check accuracy
def validate_accuracy(loader, model):
    num_correct = 0
    num_samples = 0
    model.eval()

    with torch.no_grad():
      # send data to cuda if possible
        for data, label in loader:
            data = data.to(device=device)
            label = label.to(device=device)

            scores = model(data)
            _, pred = scores.max(dim=1)
            num_correct += (pred == label).sum() # torch.sum(pred == v_label).item() also can
            num_samples += pred.size(0)

        print(f"Accuracy: {float(num_correct)/float(num_samples):.4f}")

    model.train()


Epoch [0], Loss 1.1294
Epoch [1], Loss 0.5993
Epoch [2], Loss 0.4258
Epoch [3], Loss 0.3040
Epoch [4], Loss 0.2272
Epoch [5], Loss 0.1680
Epoch [6], Loss 0.1316
Epoch [7], Loss 0.1097
Epoch [8], Loss 0.0989
Epoch [9], Loss 0.0839
Epoch [10], Loss 0.0759
Epoch [11], Loss 0.0770
Epoch [12], Loss 0.0628
Epoch [13], Loss 0.0634
Epoch [14], Loss 0.0568
Epoch [15], Loss 0.0519
Epoch [16], Loss 0.0485
Epoch [17], Loss 0.0482
Epoch [18], Loss 0.0458
Epoch [19], Loss 0.0455
CPU times: user 7min 43s, sys: 2min 2s, total: 9min 46s
Wall time: 9min 52s


In [ ]:
validate_accuracy(train_loader, model)

Accuracy: 0.9894


In [ ]:
validate_accuracy(val_loader, model)

Accuracy: 0.8401


In [ ]:
torch.save(model.state_dict(), 'cifar10_pretrainedvgg16.pth')